# Option Selection

### Library

In [1]:
import urllib, json
import multiprocessing, threading
import pandas as pd
import concurrent.futures
from functools import partial
from time import time
from datetime import date
from contextlib import contextmanager

import requests
from yahoo_fin import options
import yfinance as yf
from wallstreet import Stock, Call, Put

from Utility.stock_utility import *

In [2]:
optionable_list = []
filtered_list = []
high_iv_list = []
min_price = 30
max_price = 200

### Function

In [3]:
def price_filter(udlying, min_price = 30, max_price = 200):
    """
    filter price between min and max
    add filtered price to filtered_list
    :param udlying: stock
    :param min_price: min price default 30
    :param max_price: max price dafult 200
    :return: None
    """
    global filtered_list
    s = Stock(udlying)
    if min_price < s.price < max_price:
        filtered_list.append(udlying)

def price_filter_multi(list_of_tickers):
    """
    filter prices with multi-threads

    :param list_of_tickers: list of tickers
    :return: None

    """
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(price_filter,list_of_tickers)

def get_tickers():
    """
    Get available tickers
    :return: optionable_list, filtered_list
    """

    global filtered_list

    ## read all ticker names
    list_of_tickers = csv_read(csv_name="all_tickers_V2.csv")
    print(list_of_tickers)
    print(len(list_of_tickers))

    ## find all available tickers
    start = time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(find_optionable_stocks,list_of_tickers)

    print(optionable_list)
    print(filtered_list)

    ## write back the results to text
    with open('optionable_list.txt', 'w') as f:
        for item in optionable_list:
            f.write("%s\n" % item)
    # print(len(optionable_list))
    # print(filtered_list)
    # print(len(filtered_list))
    print(f'Time taken: {time() - start}')

    return optionable_list, filtered_list

def get_volatility(ticker="AAPL"):
    """
    Get stock's volatiliy
    :param ticker: ticker name
    :return: list of json volatility data
    """
    url = "https://www.alphaquery.com/data/option-statistic-chart"
    params = {
        "ticker" : ticker,
        "perType" : "30-Day",
        "identifier" : "iv-call"
    }
    r = requests.get(url=url,params=params)
    iv = r.json()
    return iv

def get_avg_volatility(ticker="AAPL", lookahead=30):
    """
    Calculate avg volatiliy
    :param ticker:  ticker name
    :param lookahead: lookahead
    :return:
    """

    iv = get_volatility(ticker)
    # get the most recent values
    iv_lookahead = iv[-lookahead:]
    #print(len(iv_lookahead))
    ivs = [k['value'] if k['value'] else 0.0 for k in iv_lookahead]

    iv_avg = sum(ivs) / lookahead
    return iv_avg, iv

def get_high_iv_list(ticker, threshold=0.8):
    """
    Add ticker larger than threshold to high_iv_list
    :param ticker: stock name
    :param threshold: threshold int
    :return: high_iv_list(list)
    """
    global high_iv_list
    avg, iv = get_avg_volatility(ticker)
    #print(avg)
    if avg > threshold:
        #print(ticker, "meet the threshold")
        high_iv_list.append(ticker)
        with open(f"{ticker}.json", 'w') as outfile:
            json.dump(iv, outfile)
    return high_iv_list


def DTE(expiration):
    """
    Find DTE from expiration
    :param expiration: expiration time (int)
    :return: number of days
    """
    exp_date = date(int(expiration[0:4]), int(expiration[5:7]), int(expiration[8:10])+1)
    today = date.today()
    delta = exp_date - today
    return (delta.days)
    
def find_score(expiration,premium,delta,strike):
    """
    find score
    :param expiration:
    :param premium:
    :param delta:
    :param strike:
    :return:
    """
    time_diff = DTE(expiration)
    K1 = 30/time_diff
    score = K1*(1-2*abs(delta))*premium*2000/strike
    return score

def find_score_each_expiration(expiration,udlying):
    """

    :param expiration:
    :param udlying:
    :return:
    """
    print(f"Processing {udlying}")
    ticker = yf.Ticker(udlying)
    opt = ticker.option_chain(expiration)
    df = opt.puts
    s_array = df[["strike","inTheMoney"]]
    indexNames = s_array[ (s_array['inTheMoney'] == True) ].index
    s_array.drop(indexNames , inplace=True)
    df=[]
    strikes=s_array[["strike"]].to_numpy()[::-1]
    Best_option_score = 0
    Best_option = []

    if len(strikes) == 0:
        return Best_option_score,Best_option

    for strike in strikes:
        option = Put(udlying, d=int(expiration[8:10]), m=int(expiration[5:7]), y=int(expiration[0:4]), strike=strike)
        premium = (2*option.price+option.bid+option.ask)/4
        delta = option.delta()
        score = int(find_score(expiration,premium,delta,strike))
        print(expiration, "on",udlying, float(strike),"put has a score", int(score))
        if score > Best_option_score:
            Best_option_score = score
            Best_option = "{} {} {} put with score {}.".format(udlying, expiration, float(strike), int(score))
        
        if abs(delta) < 0.1 or  premium<0.02*strike:
            return Best_option_score,Best_option

def multi_find_score (udlying):
    """

    :param udlying:
    :return:
    """
    print(udlying)
    ticker = yf.Ticker(udlying)
    expirations = ticker.options
    print(expirations)
    results = list()


    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []

        for expi in expirations[:5]:
            futures.append(executor.submit(partial(find_score_each_expiration,udlying=udlying), expi))

        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())
    print("Best option overall:",max(results)[1])

In [4]:
def multi_find_score_multiprocess(udlying,processes=None):
    print(udlying)
    ticker = yf.Ticker(udlying)
    expirations = ticker.options
    print(expirations)
    if not processes:
        with multiprocessing.Pool(processes=processes) as pool:
            results = pool.map(partial(find_score_each_expiration,udlying=udlying),expirations[:5])
            print(print("Best option overall:",max(results)[1]))
    else:
        with multiprocessing.Pool() as pool:
            results = pool.map(partial(find_score_each_expiration,udlying=udlying),expirations[:5])
            print(print("Best option overall:",max(results)[1]))

## Testing Option Selection

In [5]:
udlying="PLTR"
print(udlying)
pd.options.mode.chained_assignment = None  # default='warn'
print('finding the best option')

PLTR
finding the best option


### Multithreads

In [6]:
## Multithreads
start = time()
multi_find_score(udlying)
print(f"Multithreads: {time()-start}")

PLTR
('2021-01-08', '2021-01-15', '2021-01-22', '2021-01-29', '2021-02-05', '2021-02-19', '2021-03-19', '2021-05-21', '2021-08-20', '2021-11-19', '2022-01-21', '2023-01-20')
Processing PLTR
Processing PLTRProcessing PLTR
Processing PLTRProcessing PLTR


2021-01-152021-01-08  on PLTR 23.5 put has a score 29
on PLTR 23.52021-01-22 put has a score 28
 on PLTR 23.5 put has a score 24
2021-01-29 on PLTR 23.5 put has a score 27
2021-01-22 on PLTR 23.0 put has a score 40
2021-01-29 on PLTR 23.0 put has a score 38
2021-01-15 on PLTR 23.0 put has a score 51
2021-01-08 on PLTR 23.0 put has a score 76
2021-01-22 on PLTR 22.5 put has a score 50
2021-01-29 on PLTR 22.5 put has a score 47
2021-01-15 on PLTR 22.5 put has a score 66
2021-01-08 on PLTR 22.5 put has a score 98
2021-01-22 on PLTR 22.0 put has a score 57
2021-01-29 on PLTR 22.0 put has a score 53
2021-01-08 on PLTR 22.0 put has a score 102
2021-01-15 on PLTR 22.0 put has a score 74
2021-01-22 on PLTR 21.5 put has a score 60
2021-01-29 on 

### MultiProcess

In [7]:
## Multithreads
## multiprocess
start = time()
multi_find_score_multiprocess(udlying)
print(f"multiprocess: {time()-start}")

PLTR
('2021-01-08', '2021-01-15', '2021-01-22', '2021-01-29', '2021-02-05', '2021-02-19', '2021-03-19', '2021-05-21', '2021-08-20', '2021-11-19', '2022-01-21', '2023-01-20')
Processing PLTRProcessing PLTRProcessing PLTRProcessing PLTRProcessing PLTR




2021-01-29 on PLTR 23.5 put has a score 27
2021-01-22 on PLTR 23.5 put has a score 24
2021-01-08 on PLTR 23.5 put has a score 29
2021-01-15 on PLTR 23.5 put has a score 28
2021-01-29 on PLTR 23.0 put has a score 38
2021-01-22 on PLTR 23.0 put has a score 40
2021-01-08 on PLTR 23.0 put has a score 76
2021-01-15 on PLTR 23.0 put has a score 51
2021-01-292021-01-222021-01-08   ononon   PLTRPLTRPLTR   22.522.5 22.5 put has a score put has a score  put has a score4750 

98
2021-01-15 on PLTR 22.5 put has a score 66
2021-01-292021-01-08  onon  PLTRPLTR  22.022.0  put has a scoreput has a score  53102

2021-01-22 on PLTR 22.0 put has a score 57
2021-01-15 on PLTR 22.0 put has a score 74
2021-01-292021-01-08  onon  PLTRPLTR  21.521.5  put has a

In [8]:
## multiprocess
start = time()
multi_find_score_multiprocess(udlying,16)
print(f"multiprocess: {time()-start}")

PLTR
('2021-01-08', '2021-01-15', '2021-01-22', '2021-01-29', '2021-02-05', '2021-02-19', '2021-03-19', '2021-05-21', '2021-08-20', '2021-11-19', '2022-01-21', '2023-01-20')
Processing PLTRProcessing PLTRProcessing PLTRProcessing PLTRProcessing PLTR




2021-01-29 on PLTR 23.5 put has a score 27
2021-01-082021-01-22  onon  PLTRPLTR  23.523.5  put has a scoreput has a score  2924

2021-01-15 on PLTR 23.5 put has a score 28
2021-01-29 on PLTR 23.0 put has a score 38
2021-01-222021-01-08  onon  PLTRPLTR  23.023.0  put has a scoreput has a score  4076

2021-01-15 on PLTR 23.0 put has a score 51
2021-01-29 on PLTR 22.5 put has a score 47
2021-01-222021-01-08  onon  PLTRPLTR  22.522.5  put has a scoreput has a score  5098

2021-01-15 on PLTR 22.5 put has a score 66
2021-01-29 on PLTR 22.0 put has a score 53
2021-01-08 on PLTR 22.0 put has a score 102
2021-01-15 on PLTR 22.0 put has a score 74
2021-01-22 on PLTR 22.0 put has a score 57
2021-01-29 on PLTR 21.5 put has a score 56
2021-01-08 on 